# Upload Results to DB

In [1]:
from utils.preprocessing import *
from utils.database import *
from datasets import load_from_disk
from tqdm import tqdm

/home/jschelb/.pyenv/versions/3.10.8/envs/mediacloud/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dataset

In [2]:
dataset = load_from_disk('../data/output/articles_processed')

In [3]:
dataset = dataset.map(lambda row: {'sort_key':
                                   str(row['id']) + str(row['role']) +
                                   str(row['chunk'])
                                   })
sorted_dataset = dataset.sort('sort_key')
sorted_dataset = sorted_dataset.remove_columns('sort_key')


Map: 100%|██████████| 651/651 [00:00<00:00, 4919.38 examples/s]


## Connect to Database

In [4]:
_, db = getConnection(use_dotenv=True)

## Update Documents in Database

In [5]:
def process_dataset(dataset):
    # Initial processing results
    processing_result = {"hero": [], "villain": [], "victim": []}
    object_id_prev = None

    for item in dataset:
        object_id = item['id']
        role = item['role']
        answer = item['Answers']

        # If the object_id changes, reset the processing_result
        if object_id_prev is not None and object_id_prev != object_id:
            yield object_id_prev, processing_result
            processing_result = {"hero": [], "villain": [], "victim": []}

        processing_result[role].append(answer)
        object_id_prev = object_id

    # Yield the final processing_result if any
    if processing_result["hero"] or processing_result["villain"] or processing_result["victim"]:
        yield object_id_prev, processing_result


# Assuming `ds` is your dataset object
unique_ids = set(dataset["id"])

# Count of unique ids
count_unique_ids = len(unique_ids)
# print(count_unique_ids)

for object_id, result in tqdm(process_dataset(dataset), total=count_unique_ids, desc="Uploading results"):
    updateProcessingResults(db, object_id, {"processing_result": result})

Uploading results: 100%|██████████| 50/50 [00:00<00:00, 108.08it/s]
